# Find the best sample position based on diffraction intensity

If the sample environment is preventing the diode from being inserted a dscan of the hex_x axis can be used to find the best horizontal position.  
Can also be used for multiple positions

In [ ]:
%matplotlib widget
import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from ipywidgets import interact, widgets
import DanMAX as DM

In [ ]:
#----------------------------------------

#fname = '/data/visitors/danmax/PROPOSAL/VISIT/raw/SAMPLE/scan-XXXX.h5'

fname = DM.getLatestScan(scan_type='ascan')

# read data form the .h5 file
with h5py.File(fname) as fh:
    im = fh['/entry/instrument/pilatus/data'][:].astype(float)
    x = fh['/entry/instrument/hex_x/value'][:]

# set masked pixels to nan
im[im<0]=np.nan
# mean detector intensity
I = np.nanmean(im,axis=(1,2))
I = (I-I.min())/(I.max()-I.min())
# mean vertical detector intensity
I_vert = np.nanmean(im[:,:,685:786],axis=(1,2))
I_vert = (I_vert-I_vert.min())/(I_vert.max()-I_vert.min())

# scan resolution in mm/point
res = np.mean(np.diff(x))
# find peaks
peaks, prop     = find_peaks(I,
                             distance=int(1/res),# minimal distance between peaks
                             height=0.02) 
peaks_v, prop_v = find_peaks(I_vert,
                             distance=int(1/res),
                             prominence=0.02) # minimal distance between peaks
expo = DM.getExposureTime(fname)
print('Predicted peak positions:')
if np.all(peaks==peaks_v):
    print('Position (mm) , Hottest pixel (cps)')
    [print(f'{x[peak]:13.3f} ,  {np.nanmax(im[peak])/expo:.1E}') for peak in peaks]
else:
    print(' Mean ,   Vertical ,  Hottest pixel (cps)')
    for i,peak in enumerate(peaks):
        peak_v = peaks_v[i]
        print(f'{x[peak]:7.3f} , {x[peak_v]:7.3f}  ,  {np.nanmax(im[peak])/expo:.1E}')
plt.figure(dpi=150)
plt.title(DM.getScan_id(fname))
plt.plot(x,I,label='Mean intensity')
plt.plot(x,I_vert,label='Mean vertical intensity')

plt.plot([x[peak] for peak in peaks],[I[peak] for peak in peaks],'ro',fillstyle='none',label='mean peak(s)')
plt.plot([x[peak] for peak in peaks_v],[I_vert[peak] for peak in peaks_v],'bo',fillstyle='none',label='vertical mean peak(s)')

for i,peak in enumerate(peaks):
    peak_v = peaks_v[i]
    plt.annotate(f'{x[peak]:.3f}',(x[peak],I[peak]),color='r')
plt.xlabel('Position (mm)')
plt.ylabel('Intensity')
plt.legend()


